In [1]:
import os
import sys

up1 = os.path.abspath('..') 
sys.path.insert(0, up1)
print(sys.path)

['/data/su_buildup/su_buildup', '', '/home/ubuntu/src/cntk/bindings/python', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [2]:

import random
import cv2, numpy as np
import pickle


from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras import regularizers
from keras.callbacks import History 
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model

from keras_data_generator.ucf101_datagenerator.generator_class import DataGenerator
import csv



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## The VGG 16 model of spatial training

    So the idea here is that we're building a VGG 16 model without the top included. Essentially we want all the feature
    learning component of the model without the fully connected layer and input layer. We will also freeze VGG 16 FL layers 
    as we simply want it to learn how to classify based on the input from the sport action recognition layer. 

In [3]:
# Parameters
params = { 'data_dir' : "/data",
          'dim': (224,224),
          'batch_size': 64,
          'n_classes': 101,
          'n_channels': 3,
          'shuffle': True}


In [4]:
def getVggModel(LR,l1v,drp,printmod=1):

    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
    x = base_model.get_layer('block5_pool').output   # collect outputs from hidden layer, Block 5
    # stitch layers to the VGG16 layers
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)

    predictions = Dense(101, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # freeze original VGG16 layers
    for i, layer in enumerate(model.layers):
        if 'block1' in layer.name or 'block2' in layer.name or 'block3' in layer.name or \
        'block4' in layer.name or 'block5' in layer.name:
            layer.trainable = False
        else:
            layer.trainable = True
            
    mypotim = SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    if (printmod==1 ):
        model.summary()
    return model

In [5]:

def getData(listPath):
# load Id list and labels 

    train_list = list()

    with open(listPath, 'r') as f:
        reader = csv.reader(f)
        train_list = list(reader)

        labels = [int(label[0].split(' ')[1]) for label in train_list ]
    IDs = [label[0].split(' ')[0] for label in train_list ]
    # IDs
    IDs = [id.split('/')[1].rstrip('.avi') for id in IDs ]

    labels = dict(zip(IDs, labels))
    return (IDs, labels)

id_labels_train = getData('../keras_data_generator/ucf101_splits/trainlist01.txt')
id_labels_test = getData('../keras_data_generator/ucf101_splits/trainlist02.txt')

training_generator = DataGenerator(*id_labels_train, **params)
validation_generator = DataGenerator(*id_labels_test, **params)


In [6]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=10):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [7]:
model=getVggModel(1e-4,1e-1,0.9)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
# Train model on dataset
mod1 = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=2, epochs=10,
                    verbose=1)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[25088,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_1/kernel/Assign = Assign[T=DT_FLOAT, _class=["loc:@dense_1/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1/kernel, dense_1/random_uniform)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_1/kernel/Assign', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()


In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# A better Image  Generator

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath):
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/data/sym_datasets/jpegs_256_1_train/',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        follow_links=True
        )

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/data/sym_datasets/jpegs_256_1_test_15_frames/',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        follow_links=True)


In [ ]:
# Train model on dataset
mod1 = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=2, epochs=40,
                    verbose=1, callbacks=callbacks)

In [ ]:
model.evaluate_generator(generator=validation_generator,  use_multiprocessing=True,
                    workers=2)